In [2]:
import numpy as np
import copy
import random
import time
import os
import pandas as pd
from scipy.io import mmread

Group: 

Caio Graça Gomes, 

Gabriel Tostes Messias Pereira.

# Weighted Minimum Vertex Cover Problem

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path_instances = '/content/drive/MyDrive/Instances'

In [5]:
!ls '/content/drive/MyDrive/Instances'

bio-celegans.mtx		 lp1.mtx
bio-diseasome.mtx		 opsahl-southernwomen.edges
bio-yeast.mtx			 road-chesapeake.mtx
bn-macaque-rhesus_brain_2.edges  rt-retweet-crawl.mtx
boyd2.mtx			 rt-retweet.mtx
ca-CSphd.mtx			 rt-twitter-copen.mtx
ca-Erdos992.mtx			 soc-dolphins.mtx
can_96.mtx			 soc-douban.mtx
ca-netscience.mtx		 soc-gplus.edges
com-youtube.edges		 soc-karate.mtx
dwt_59.mtx			 tech-as-caida2007.mtx
GD95_c.mtx			 web-edu.mtx
gent113.mtx			 web-google.mtx
ia-email-EU.mtx			 web-indochina-2004.mtx
ia-enron-only.mtx		 web-polblogs.mtx
ia-reality.mtx			 web-webbase-2001.mtx
inf-contiguous-usa.edges


Given an undirected graph $G = (V, E)$, and a cost function $c: V \rightarrow \mathbb{R}^{*}_{+}$, find a subset $V' \subseteq V$ such that, if $uv \in E$, then $u \in V' \lor e \in V'$, moreover $S = \sum^{}_{v \in V'} c(v)$ is minimal.

A more convenient way to formulate the problem would be:

Minimize $\sum_{v \in V} c(v) x_v$

Subject to $x_u + x_v \geq 1 \forall \{u, v\} \in E$

and $x_v \in \{0, 1\} \forall v \in V$

Therefore, the solution can be written as a boolean array $x(G) = [x_1, x_2, ..., x_{|V|}]$.

### Data Structure Definition

In [5]:
class solution():
  def __init__(self, num_vertices):
    self.array = [0 for i in range(num_vertices)]
    self.prize = 0
  def update(self, weights):
    total_prize = 0
    for i in range(len(weights)):
      total_prize += self.array[i] * weights[i]
    self.prize = total_prize

### Auxiliary Functions

In [6]:
def is_feasible(solution, edges):
  for edge in edges:
    if not solution.array[edge[0]] and not solution.array[edge[1]]: return False
  return True

## Genetic Algorithm Implementation

In [7]:
class sga():
    
    def __init__(self, weights, edges, population_size = 20, recombinate_probability = .6, mutation_probability = .03, elitism = .1, generations = 30):
        self.num_vertices = len(weights)
        self.weights = copy.deepcopy(weights)
        self.edges = copy.deepcopy(edges)
        self.deg = np.zeros(len(weights))
        self.adj = {i: [] for i in range(len(weights))}

        for edge in self.edges:
          self.deg[edge[0]]+=1
          self.adj[edge[0]].append(edge[1])
          if edge[0] != edge[1]:
            self.deg[edge[1]]+=1
            self.adj[edge[1]].append(edge[0])

        
        self.population_size = population_size
        self.recombinate_probability = recombinate_probability
        self.mutation_probability = mutation_probability
        self.elitism = elitism
        self.max_generations = generations

        self.alpha = np.sqrt(population_size)

        self.population = []

        self.best_solution = solution(self.num_vertices)
        self.best_gen = 1

        self.ps = 0
        self.c = 0
        self.m = 0
        self.r = 0
        self.h = 0
        self.ng = 0
        self.b = 0
    
    # População inicial é gerada usando um método aleatório que retorna soluções válidas.
    def generate_initial_population(self):
        self.population = []
        for i in range(self.population_size):
            self.population.append(solution(self.num_vertices))
        self.repair(self.population)

    
    # Método para seleção de pais e da próxima geração, por adaptabilidade(usando as recompensas).
    def parent_selection(self):
        
        fitness = []
        cumulated_prize = 0
        
        for i in range(self.population_size):
          fit = np.exp(-self.alpha * self.population[i].prize)
          fitness.append(fit)
          cumulated_prize += fit
        
        selection_probabilities = fitness / cumulated_prize
        
        selector = []
        cumulated_probability = 0
        
        for i in range(self.population_size):
            cumulated_probability += selection_probabilities[i]
            selector.append(cumulated_probability)
            
        parents = []
        
        for i in range(self.population_size):
            
            select = np.random.uniform(0, 1)
            
            not_found = True
            selected = 0
            
            while(selected < self.population_size and not_found):
                
                if select < selector[selected]:
                    parents.append(self.population[selected])
                    not_found = False
                    
                selected += 1
        
        return parents

    def mutation(self, children):
        u01 = np.random.uniform(0,1)
        if u01 < 0.2: 
          for child in children:
              for i in range(len(child.array)):
                if np.random.uniform(0, 1) < self.mutation_probability:
                  child.array[i] = int(not child.array[i])
              child.update(self.weights)
        else:
          for child in children:
            p = np.zeros(len(child.array))
            for i in range(len(child.array)):
              if child.array[i]:
                p[i] = self.weights[i] / (self.deg[i]+0.001)
            if p.sum()==0: return children

            p = p/p.sum()

            for i in range(2):
              dr = np.random.choice(np.arange(len(child.array)), 1, p = p)[0]
              if child.array[dr]: child.array[dr] = 0
            child.update(self.weights)
        return children

    def repair(self, children):
      for child in children:
        edg = []
        for e in self.edges:
          if not child.array[e[0]] and not child.array[e[1]]: edg.append(e)
        edg = np.array(edg)
        np.random.shuffle(edg)
        for e in edg:
          if child.array[e[0]] or child.array[e[1]]:  continue
          p0 = self.weights[e[0]]/(self.deg[e[0]]+1)
          p1 = self.weights[e[1]]/(self.deg[e[1]]+1)
          u01 = random.uniform(0,1)
          if u01 < p0/(p0+p1): child.array[e[1]]=1
          else: child.array[e[0]]=1
        child.update(self.weights)

      return children
      
    def eliminate_redundant(self, redundant, elem):
      for u in self.adj[elem]:
        if u in redundant: redundant.remove(u)
      if elem in redundant: redundant.remove(elem)


    def heuristic(self, children):
      for child in children:
        aux = time.time()
        redundant = []
        cnt = np.zeros(len(child.array))

        for i in range(len(child.array)):
          if not child.array[i]: continue
          ct = 0
          for j in self.adj[i]:
            if child.array[j]: ct+=1
            else: break
          if ct>0 and ct==self.deg[i] and (i not in self.adj[i]): redundant.append(i)

        while len(redundant):
          p = np.zeros(len(redundant))
          for i in range(len(redundant)):
            p[i] = self.weights[i]/(self.deg[i]+0.001)
          p = np.exp(2*np.array(p))
          p = p/p.sum()

          elem = np.random.choice(redundant, 1, p = p)[0]
          child.array[elem] = 0
          self.eliminate_redundant(redundant, elem)

        child.update(self.weights)

      return children
                
    # def parent_selection(self):
    #   parents = []
    #   for i in range(self.population_size):
    #     parent1 = self.population[random.randint(0, self.population_size-1)]
    #     parent2 = self.population[random.randint(0, self.population_size-1)]
    #     parent3 = self.population[random.randint(0, self.population_size-1)]
    #     if parent1.prize <= parent2.prize and parent1.prize <= parent3.prize:
    #       parents.append(parent1)
    #     elif parent2.prize <= parent2.prize and parent2.prize <= parent3.prize:
    #       parents.append(parent2)
    #     elif parent3.prize <= parent1.prize and parent3.prize <= parent2.prize:
    #       parents.append(parent3)
    #   return parents
    

    # Método de recombinação, crossover de dois pontos.
    def recombinate(self, parent1, parent2):
        
        child1 = copy.deepcopy(parent1)
        child2 = copy.deepcopy(parent2)
        
        index1 = np.random.randint(self.num_vertices)
        index2 = np.random.randint(self.num_vertices)
        
        index1, index2 = np.sort([index1, index2])

        for i in range(index1, index2):
          child1.array[i] = parent2.array[i]
        for i in range(index1, index2):
          child2.array[i] = parent1.array[i]

        child1.update(self.weights)
        child2.update(self.weights)

        return child1, child2
    
    def crossover(self, parents):
        
        np.random.shuffle(parents)
        
        children = []
        
        for i in range(self.population_size // 2):
            
            children.append(parents[2*i])
            children.append(parents[2*i+1])
            
            if np.random.uniform(0, 1) < self.recombinate_probability:
                children[2*i], children[2*i + 1] = self.recombinate(parents[2*i], parents[2*i+1])
        
        return children
        
        
    def next_generation_selection(self, parents, children):
        
        next_gen = []
        
        parents.sort(key = lambda x: x.prize)
        children.sort(key = lambda x: x.prize)

        for i in range(self.population_size):
          if i < self.elitism * self.population_size:
            next_gen.append(parents[i])
          else:
            next_gen.append(children[int(i - self.elitism * self.population_size)])

        return next_gen
        
    # Resolução do algoritmo genético, adotou-se como critério de parada alcançar 50 iterações
    def solve(self):
        
        self.generate_initial_population()

        self.best_solution = copy.deepcopy(self.population[0])

        stop_criterion = False
        counter = 0
        
        while(not stop_criterion):
            # print(f'Counter : {counter} : {time.time()}')
            counter += 1
            
            aux = time.time()
            parents = self.parent_selection()
            self.ps += time.time() - aux
            
            aux = time.time()
            children = self.crossover(parents)
            self.c += time.time() - aux
            aux = time.time()
            children = self.mutation(children)
            self.m += time.time() - aux
            aux = time.time()
            children = self.repair(children)
            self.r += time.time() - aux
            aux = time.time()
            children = self.heuristic(children)
            self.h += time.time() - aux

            aux = time.time()
            self.population = self.next_generation_selection(parents, children)
            self.ng += time.time() - aux

            aux = time.time()
            for individual in self.population:
              if individual.prize < self.best_solution.prize:
                self.best_solution = copy.deepcopy(individual)
                s.best_gen = counter
            self.b += time.time() - aux

            if(counter > self.max_generations): 
                stop_criterion = True

In [8]:
class flipheuristic():

  def __init__(self, w, e):
    self.w_ = copy.deepcopy(w)
    self.e_ = copy.deepcopy(e)
    random.shuffle(self.e_)
    self.taken = list(np.zeros(len(w)))
    self.ans = 0

  def getedge(self):
    while len(self.e_):
      e = self.e_.pop(0)
      if not self.taken[e[0]] and not self.taken[e[1]]: return e
    return None
    
  def step(self):
    e = self.getedge()
    if e is None: return 0
    w = self.w_
    u = e[0]
    v = e[1]
    if np.random.rand() <= w[u]/(w[u]+w[v]):
      self.taken[v] = 1
      self.ans+=w[v]
    else:
      self.taken[u] = 1
      self.ans+=w[u]
    return 1


  def run(self):
    while self.step(): pass
    return self.ans, self.taken

# Teste

In [9]:
def graph(num_vertices, maxweight=11, name='bio-celegans'):
  p = .3
  edges = []
  for i in range(num_vertices):
    for j in range(i+1, num_vertices):
      if np.random.uniform(0, 1) < p:
        edges.append([i, j])

  weights = np.random.randint(1, maxweight, size=num_vertices)
  arx = open(f'{name}.mwvc', "a")
  arx.write(f'p edge {num_vertices} {len(edges)}\n')
  for i in range(1,num_vertices+1):
    arx.write(f'v {i} {weights[i-1]}\n')
  for i in edges:
    arx.write(f'e {i[0]+1} {i[1]+1}\n')
  return edges, weights

In [ ]:
!ls /content/drive/MyDrive/Instances

bio-celegans.mtx		 lp1.mtx
bio-diseasome.mtx		 opsahl-southernwomen.edges
bio-yeast.mtx			 road-chesapeake.mtx
bn-macaque-rhesus_brain_2.edges  rt-retweet-crawl.mtx
boyd2.mtx			 rt-retweet.mtx
ca-CSphd.mtx			 rt-twitter-copen.mtx
ca-Erdos992.mtx			 soc-dolphins.mtx
can_96.mtx			 soc-douban.mtx
ca-netscience.mtx		 soc-gplus.edges
com-youtube.edges		 soc-karate.mtx
dwt_59.mtx			 tech-as-caida2007.mtx
GD95_c.mtx			 web-edu.mtx
gent113.mtx			 web-google.mtx
ia-email-EU.mtx			 web-indochina-2004.mtx
ia-enron-only.mtx		 web-polblogs.mtx
ia-reality.mtx			 web-webbase-2001.mtx
inf-contiguous-usa.edges


In [10]:
def toWE_mtx(name):
  end = f'/content/drive/MyDrive/Instances/{name}.mtx'
  a = mmread(end).toarray()
  w = [(i+1)%200 for i in range(1, len(a)+1)]
  e = []
  for i in range(len(a)):
    for j in range(i,len(a)):
      if a[i][j]: e.append([i,j])
  return w,e

In [26]:
w,e = toWE_mtx('ca-netscience')

In [27]:
edges = e
weights = w
norm_w = np.linalg.norm(w)
normalized_w = w / norm_w

In [28]:
num_iterations = 3

results = [0 for i in range(num_iterations)]
solutions = [[] for i in range(num_iterations)]
results_flip = [0 for i in range(num_iterations)]
solutions_flip = [[] for i in range(num_iterations)]

for i in range(num_iterations):
  print(i, end = ' ')
  flip = flipheuristic(w,e)
  results_flip[i], solutions_flip[i] = flip.run()
  # print(time.time())
  s = sga(normalized_w, e, population_size = 100, generations=100)
  s.solve()
  results[i] = int(round(s.best_solution.prize * norm_w))
  print('Prize: ', results[i], '/ Is feasible: ', is_feasible(s.best_solution, e), '/ Best generation:', s.best_gen)
  # print('Times:\n Parent Selection: ', s.ps, 'Crossover: ', s.c, 'Mutation: ', s.m, 'Repair: ', s.r, 'Heuristic: ', s.h, 'Find best: ', s.b)

print('\nGA: ', results)
print('Média: ', np.mean(results))
print('Mediana: ', np.median(results))
print('Mínimo: ', min(results))

print('\nFLIP: ', results_flip)
print('Média: ', np.mean(results_flip))
print('Mediana: ', np.median(results_flip))
print('Mínimo: ', min(results_flip))

0 Prize:  19305 / Is feasible:  True / Best generation: 89
1 Prize:  19305 / Is feasible:  True / Best generation: 98
2 Prize:  19306 / Is feasible:  True / Best generation: 81

GA:  [19305, 19305, 19306]
Média:  19305.333333333332
Mediana:  19305.0
Mínimo:  19305

FLIP:  [22061, 22849, 21952]
Média:  22287.333333333332
Mediana:  22061.0
Mínimo:  21952


In [ ]:
huge_files = []

In [ ]:
os.path.getsize(path_instances+'/soc-dolphins.mtx')

3056

In [9]:
for filename in os.listdir(path_instances):
  if filename.endswith('.mtx') and os.path.getsize(path_instances + '/' + filename) < 1e6:
    w, e = toWE_mtx(filename[:-4])
    print(filename, len(e))

bio-celegans.mtx 2025
rt-twitter-copen.mtx 1029
soc-karate.mtx 78
soc-dolphins.mtx 159
bio-diseasome.mtx 1188
bio-yeast.mtx 1948
web-edu.mtx 6474
ia-reality.mtx 7680
tech-as-caida2007.mtx 53381
web-webbase-2001.mtx 25593
web-google.mtx 2773
dwt_59.mtx 163
ia-email-EU.mtx 54397
road-chesapeake.mtx 170
can_96.mtx 432
gent113.mtx 312
rt-retweet.mtx 117
ia-enron-only.mtx 623
web-indochina-2004.mtx 47606
web-polblogs.mtx 2280
GD95_c.mtx 143
ca-netscience.mtx 914
ca-CSphd.mtx 1094
ca-Erdos992.mtx 7515


In [11]:
for filename in os.listdir(path_instances):
  if os.path.getsize(path_instances + '/' + filename) < 1e6:
    if filename.endswith('.mtx'):
        print(filename[:-4])
        
        try:
          w,e = toWE_mtx(filename[:-4])
          
          edges = e
          weights = w
          norm_w = np.linalg.norm(w)
          normalized_w = w / norm_w

          start = time.time()
          flip = flipheuristic(w,e)
          result_flip, solution_flip_array = flip.run()
          solution_flip = solution(len(w))
          solution_flip.array = solution_flip_array
          solution_flip.prize = result_flip
          print('Flip Time: ', time.time()-start, ' / Flip Prize: ', result_flip, ' / Is feasible: ', is_feasible(solution_flip, e))
          # start = time.time()
          # s = sga(normalized_w, e, population_size = 100, generations=100)
          # s.solve()
          # result_ga = int(round(s.best_solution.prize * norm_w))
          # print('GA Time: ', time.time() - start, ' / GA Prize: ', result_ga, ' / Is feasible: ', is_feasible(s.best_solution, e), ' / Best generation:', s.best_gen)
        except:
          print('Error')

    elif filename.endswith('.edges'):
      print('edges')

bio-celegans
Flip Time:  0.00940394401550293  / Flip Prize:  25627  / Is feasible:  True
rt-twitter-copen
Flip Time:  0.005272865295410156  / Flip Prize:  28053  / Is feasible:  True
soc-karate
Flip Time:  0.00048065185546875  / Flip Prize:  251  / Is feasible:  True
soc-dolphins
Flip Time:  0.0008218288421630859  / Flip Prize:  1419  / Is feasible:  True
bio-diseasome
Flip Time:  0.005753993988037109  / Flip Prize:  26537  / Is feasible:  True
bio-yeast
Flip Time:  0.01060628890991211  / Flip Prize:  52717  / Is feasible:  True
web-edu
Flip Time:  0.038374900817871094  / Flip Prize:  171058  / Is feasible:  True
ia-reality
Flip Time:  0.04550290107727051  / Flip Prize:  7121  / Is feasible:  True
edges
tech-as-caida2007
Flip Time:  0.5954957008361816  / Flip Prize:  475358  / Is feasible:  True
web-webbase-2001
Flip Time:  0.25968074798583984  / Flip Prize:  335569  / Is feasible:  True
web-google
Flip Time:  0.014513492584228516  / Flip Prize:  53673  / Is feasible:  True
edges
dwt_5